<a href="https://colab.research.google.com/github/sirakik/japan_graph/blob/master/%E8%88%AA%E7%A9%BA%E3%83%87%E3%83%BC%E3%82%BF%E6%95%B4%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

国土交通省　国土数値情報｜空港間流通量　https://nlftp.mlit.go.jp/ksj/gml/datalist/KsjTmplt-S10b-v1_1.html  
平成26年度データ

手作業で, 国際便のデータを削除  
手作業で, 空港座標を削除

In [ ]:
<ksj:BetAport gml:id="aport_1">
  <ksj:depAport>東京</ksj:depAport> # 出発空港
  <ksj:arrAport>新千歳</ksj:arrAport> # 到着空港
  <ksj:distAirRt>894</ksj:distAirRt>  # 運行距離(km)
  <ksj:frequency>381</ksj:frequency> # 頻度 週当たりの本数
  <ksj:numOfPass>4391501</ksj:numOfPass> # 旅客数 
  <ksj:pasVolume>6448411</ksj:pasVolume> # 運搬可能旅客数
  <ksj:numOfCargo>87472</ksj:numOfCargo> # 貨物量
  <ksj:airportLine xlink:href="#cv1"/>
</ksj:BetAport>

In [5]:
airport={0:('稚内', '利尻', '旭川', '紋別', '女満別', '中標津', '釧路', '帯広', '札幌丘珠', '函館', '新千歳', '奥尻'), 1:('青森', '三沢'), 2:('花巻'), 3:('仙台'), 4:('秋田', '大館能代'), 5:('庄内', '山形'), 6:('福島'), 7:('茨城'), 8:(), 9:(), 10:(), 11:('成田'), 12:('東京', '調布', '大島', '新島', '神津島', '三宅島', '八丈島'), 13:(), 14:('新潟'), 15:('富山'), 16:('小松', '能登'), 17:(), 18:(), 19:('松本'), 20:(), 21:('静岡'), 22:('名古屋', '中部'), 23:(), 24:(), 25:(), 26:('大阪', '関西'), 27:('神戸', '但馬'), 28:(), 29:('南紀白浜'), 30:('鳥取', '米子'), 31:('出雲', '隠岐', '石見'), 32:('岡山'), 33:('広島'), 34:('山口宇部', '岩国'), 35:('徳島'), 36:('高松'), 37:('松山'), 38:('高知'), 39:('北九州', '福岡'), 40:('佐賀', '対馬', '壱岐', '福江'), 41:('長崎'), 42:('熊本', '天草'), 43:('大分'), 44:('宮崎'), 45:('鹿児島', '種子島', '屋久島', '奄美', '喜界', '徳之島', '沖永良部', '与論'), 46:('那覇', '粟国', '久米島', '北大東', '南大東', '宮古', '下地島', '多良間', '石垣', '与那国')}